In [1]:
import pandas as pd
import noor  # Registers the .indicators accessor on Series

# Example price data
df = pd.DataFrame({
    "high":  [10, 11, 12, 13, 12, 11, 12],
    "low":   [8, 9, 10, 11, 10, 9, 10],
    "close": [9, 10, 11, 12, 11, 10, 11]
})

# Test indicators using the Series accessor
print("SMA:")
print(df["close"].indicators.sma(window_size=3))

print("\nRSI:")
print(df["close"].indicators.rsi(window_size=3))

print("\nMACD:")
print(df["close"].indicators.macd())

print("\nBollinger Bands:")
print(df["close"].indicators.bollinger_bands(window_size=3))

print("\nATR:")
print(df["close"].indicators.atr(high=df["high"], low=df["low"], window_size=3))

SMA:
0          NaN
1          NaN
2    10.000000
3    11.000000
4    11.333333
5    11.000000
6    10.666667
Name: close, dtype: float64

RSI:
0      NaN
1      NaN
2      NaN
3    100.0
4     50.0
5     25.0
6     62.5
Name: close, dtype: float64

MACD:
       macd    signal  histogram
0  0.000000  0.000000   0.000000
1  0.079772  0.015954   0.063818
2  0.221135  0.056990   0.164144
3  0.409141  0.127420   0.281720
4  0.472004  0.196337   0.275667
5  0.436105  0.244291   0.191814
6  0.482782  0.291989   0.190793

Bollinger Bands:
   upper_band  middle_band  lower_band
0         NaN          NaN         NaN
1         NaN          NaN         NaN
2   12.000000    10.000000    8.000000
3   13.000000    11.000000    9.000000
4   12.488034    11.333333   10.178633
5   13.000000    11.000000    9.000000
6   11.821367    10.666667    9.511966

ATR:
0    NaN
1    NaN
2    2.0
3    2.0
4    2.0
5    2.0
6    2.0
dtype: float64
